<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course2/week3/TensorFlow_Tutorial_v3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텐서플로우 튜토리얼 #

이번 주차 프로그래밍 과제에 오신것을 환영합니다! 지금까지, 여러분은 `numpy` 라이브러리만 가지고 신경망을 만들었습니다. 이제 지금부터는 더 복잡한 인공신경망을 보다 쉽게 만들 수 있는 딥러닝 프레임워크를 다뤄봅시다. TensorFlow, PaddlePaddle, Torch, Caffe, Keras 등의 많은 프레임워크들은 머신 러닝 개발 속도를 더욱 빠르게 만들어줍니다. 이 프레임워크들은 문서화 역시 매우 잘 되어 있어서, 막히는 부분이 있을 때마다 언제든지 읽고 문제를 해결할 수 있도록 되어 있습니다. 이번 과제에서 여러분들은 텐서플로를 활용하여 아래의 것들을 하실 수 있습니다.
- 파라미터 초기화
- session 열고 시작하기
- 알고리즘 훈련하기
- 인공 신경망 구현하기

인공지능 프레임워크는 코딩 시간을 줄여줄 뿐만 아니라, 코드의 속도를 눂여주는 최적화를 해주기도 합니다.

## 1. 텐서플로우 라이브러리 간단히 살펴보기 ##

시작하기에 앞서, 아래의 라이브러리들을 불러옵니다.

In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

%matplotlib inline
np.random.seed(1)

이제 라이브러리를 불러왔으니, 다양한 응용 프로그램들을 살펴보겠습니다. 다음은 하나의 훈련 데이터에 대한 손실을 구하는 함수입니다.

$$loss = \mathcal{L}(\hat{y}, y) = (\hat y^{(i)} - y^{(i)})^2 \tag{1}$$

In [ ]:
y_hat = tf.constant(36, name='y_hat')            # Define y_hat constant. Set to 36.
y = tf.constant(39, name='y')                    # Define y. Set to 39

loss = tf.Variable((y - y_hat)**2, name='loss')  # Create a variable for the loss

init = tf.global_variables_initializer()         # When init is run later (session.run(init)),
                                                 # the loss variable will be initialized and ready to be computed
with tf.Session() as session:                    # Create a session and print the output
    session.run(init)                            # Initializes the variables
    print(session.run(loss))                     # Prints the loss

텐서플로우에서 프로그램을 개발하고 실행하는 단계는 다음과 같습니다.

1. 아직 실행/평가되지 않은 Tensor(변수)를 만듭니다.
2. 해당 Tensor간의 연산을 구현합니다.
3. Tensor를 초기화합니다.
4. Session을 생성합니다.
5. Session을 실행합니다. 이 Session은 위에 작성한 연산을 수행합니다.

그러므로, `loss` 라는 변수를 만들 때 우리는 단순히 손실을 다른 수량의 함수로 정의했지만 그 값을 평가하지는 않았습니다. 이를 평가하려면 `init = tf.global_variables_initializer()`를 실행해야 합니다. 이 함수의 결과로 `loss` 변수가 초기화되고, 마지막 줄에서 `loss`값을 평가하고 출력할 수 있습니다.

보다 쉬운 예를 살펴보겠습니다. 아래 코드 블록을 실행해보세요.

In [ ]:
a = tf.constant(2)
b = tf.constant(10)
c = tf.multiply(a,b)
print(c)

예상대로 식의 최종 결과값 20을 찾을 수 없습니다. 출력값 `c`는 shape 속성이 없고, "int32" 타입의 텐서입니다. 일련의 작업은 'computation graph'에 계산 식을 넣은 것에 불과하고, 당연히 이 값은 아직 계산되지 않았습니다. 실제로 곱해진 결과를 확인하기 위해선, session을 만들고 이를 실행해야 합니다.

In [ ]:
sess = tf.Session()
print(sess.run(c))

이제 결과값이 출력됩니다. 지금까지의 과정을 요약하자면, **변수들을 초기화하고, session을 돌려서 그 내부의 연산을 실행시키세요**.

다음으로, `placeholder`에 대해서 알아봅니다. `placeholder`는 나중에 값을 지정할 수 있는 객체입니다. placeholder에 대한 값을 지정하려면, "feed dictionary"(`feed_dict` 변수)를 사용하여 값을 전달할 수 있습니다. 아래 코드를 실행시켜 에서 `x`에 대한 placeholder를 만들어보세요. 이를 통해 추후 세션을 실행할 때 값을 전달할 수 있습니다. 

In [ ]:
# Change the value of x in the feed_dict

x = tf.placeholder(tf.int64, name = 'x')
print(sess.run(2 * x, feed_dict = {x: 3}))
sess.close()

`x` 변수를 처음으로 정의했을 때는 해당 변수의 값이 아직 구체적으로 명시되지 않은 상태입니다. placeholder는 단순히 나중에(구체적으로는 세션이 실행될 때) 데이터를 할당할 수 있도록 하는 변수입니다. 이렇게 세션이 실행될 때 placeholder에 데이터를 할당하는 작업을 **feed data** 라고 부릅니다.

계산에 필요한 작업을 지정할 때 텐서플로우에 계산 그래프를 구성하는 방법을 알려줍니다. 계산 그래프에는 나중에 값을 지정할 일부 placeholder가 있을 수 있습니다. 마지막으로 세션을 실행할 때 TensorFlow에 계산 그래프를 실행하도록 지시합니다. 

### 1-1. Linear function ###

$Y = WX + b$ 의 계산 공식을 가지고 프로그래밍 과제를 시작해봅시다, $W$와 $X$는 랜덤 배열이고, $b$는 랜덤 벡터입니다.

**연습 문제** $WX+b$ 식을 계산하세요. $W, X, b$는 임의의 정규분포에서 추출됩니다. $W$는 (4,3) 형태의 배열이고, $X$는 (3,1)의 배열입니다. $b$는 (4,1)의 벡터입니다. 아래 코드는 3,1의 형태를 가지는 $X$를 초기화하는 코드입니다.
```python
X = tf.constant(np.random.randn(3,1), name = "X")
```

아래 함수를 찾아보면 도움이 될겁니다.
- `tf.matmul(..., ...)` : 행렬곱
- `tf.add(..., ...)` : 덧셈
- `np.random.randn(..)`: 무작위로 초기화

In [ ]:
# GRADED FUNCTION: linear_function

def linear_function():
    """
    Implements a linear function: 
            Initializes X to be a random tensor of shape (3,1)
            Initializes W to be a random tensor of shape (4,3)
            Initializes b to be a random tensor of shape (4,1)
    Returns: 
    result -- runs the session for Y = WX + b 
    """
    
    np.random.seed(1)
    
    """
    Note, to ensure that the "random" numbers generated match the expected results,
    please create the variables in the order given in the starting code below.
    (Do not re-arrange the order).
    """
    ### START CODE HERE ### (4 lines of code)
    X = None
    W = None
    b = None
    Y = None
    ### END CODE HERE ### 
    
    # Create the session using tf.Session() and run it with sess.run(...) on the variable you want to calculate
    
    ### START CODE HERE ###
    sess = None
    result = None
    ### END CODE HERE ### 
    
    # close the session 
    sess.close()

    return result

In [ ]:
print( "result = \n" + str(linear_function()))

**모범 답안**: 

```
result = 
[[-2.15657382]
 [ 2.95891446]
 [-1.08926781]
 [-0.84538042]]
```

### 1-2. 시그모이드 함수 계산하기 ###

텐서플로우를 활용한 첫 번째 선형 함수를 구현하는데 성공했습니다. 텐서플로우는 인공신경망 개발에 주로 사용되는 다양한 함수들을 제공합니다. 그 대표적인 에시로는 `tf.sigmoid`와 `tf.softmax`가 있습니다. 이번 연습문제에서는 시그모이드 함수를 계산하는 코드를 작성해보겠습니다.

이번 과제를 수행할때는 placeholder 변수인 `x`를 활용합니다. 세션이 시작될 때 `feed_dict`를 통해서 입력 `z`를 전달해야 합니다. 이번 연습에서는,
1. placeholder를 만들고
2. 시그모이드 함수를 계산하기 위한 일련의 계산 공식을 정의하고 (`tf.sigmoid` 함수를 사용합니다)
3. 세션을 실행합니다.

**연습 문제**: 아래의 시그모이드 함수를 완성하세요. 아래 함수들을 사용해서 구현해야 합니다.
- `tf.placeholder(tf.float32, name="...")`
- `tf.sigmoid(...)`
- `sess.run(..., feed_dict = {x: z})`

추가로 텐서플로우 세션을 만들고 실행하는데 두 가지 방법이 있습니다.

**방법 1**:
```python
sess = tf.Session()
# Run the variables initialization (if needed), run the operations
result = sess.run(..., feed_dict = {...})
sess.close() # Close the session
```
**방법 2:**
```python
with tf.Session() as sess: 
    # run the variables initialization (if needed), run the operations
    result = sess.run(..., feed_dict = {...})
    # This takes care of closing the session for you :)
```



In [ ]:
# GRADED FUNCTION: sigmoid

def sigmoid(z):
    """
    Computes the sigmoid of z
    
    Arguments:
    z -- input value, scalar or vector
    
    Returns: 
    results -- the sigmoid of z
    """
    
    ### START CODE HERE ### ( approx. 4 lines of code)
    # Create a placeholder for x. Name it 'x'.
    x = None

    # compute sigmoid(x)
    sigmoid = None

    # Create a session, and run it. Please use the method 2 explained above. 
    # You should use a feed_dict to pass z's value to x. 
    None
        # Run session and call the output "result"
        result = None

    ### END CODE HERE ###
    
    return result

In [ ]:
print ("sigmoid(0) = " + str(sigmoid(0)))
print ("sigmoid(12) = " + str(sigmoid(12)))

**모범 답안**: 
<table> 
<tr> 
<td>
<b>sigmoid(0)</b>
</td>
<td>
0.5
</td>
</tr>
<tr> 
<td>
<b>sigmoid(12)</b>
</td>
<td>
0.999994
</td>
</tr> 

</table> 

**요약하자면,**
1. placeholder를 생성합니다.
2. 계산하고자 하는 공식에 맞게 계산 그래프(computation graph)를 정의합니다.
3. 세션을 생성합니다.
4. 세션을 실행하고, feed dictionary를 사용해서 placeholder에 명시적으로 값을 지정해줍니다.

### 1-3. Cost 계산하기 ###

텐서플로우에는 시그모이드 뿐만 아니라 인공 신경망의 비용함수 역시 빌트인 함수로 개발되어 있습니다. 따라서 아래 공식을 구현하는 코드를 직접 작성할 필요 없이 빌트인 함수를 호출하면 됩니다.

$a^{[2](i)}$ and $y^{(i)}$ for i=1...m: 
$$ J = - \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log a^{ [2] (i)} + (1-y^{(i)})\log (1-a^{ [2] (i)} )\large )\small\tag{2}$$

이 복잡한 공식을 텐서플로우에선 한줄의 코드로 호출할 수 있습니다.

**연습 문제** : 크로스 엔트로피 손실함수를 구현해보세요. 텐서플로우에서 호출해야하는 함수는 다음과 같습니다.
```python
tf.nn.sigmoid_cross_entropy_with_logits(logits = ..., labels = ...)
```

입력값 `z`에 대해서, 활성화 변수인 `a`를 얻기 위한 시그모이드 활성화 변수를 얻고, 크로스 엔트로피 비용함수 $J$를 계산합니다. 일련의 모든 과정은 `tf.nn.sigmoid_cross_entropy_with_logits` 함수 호출 한번에 끝납니다. 위 함수는 아래 공식을 계산하는 함수입니다.

In [ ]:
# GRADED FUNCTION: cost

def cost(logits, labels):
    """
    Computes the cost using the sigmoid cross entropy
    
    Arguments:
    logits -- vector containing z, output of the last linear unit (before the final sigmoid activation)
    labels -- vector of labels y (1 or 0) 
    
    Note: What we've been calling "z" and "y" in this class are respectively called "logits" and "labels" 
    in the TensorFlow documentation. So logits will feed into z, and labels into y. 
    
    Returns:
    cost -- runs the session of the cost (formula (2))
    """
    
    ### START CODE HERE ### 
    
    # Create the placeholders for "logits" (z) and "labels" (y) (approx. 2 lines)
    z = None
    y = None
    
    # Use the loss function (approx. 1 line)
    cost = None
    
    # Create a session (approx. 1 line). See method 1 above.
    sess = None
    
    # Run the session (approx. 1 line).
    cost = None
    
    # Close the session (approx. 1 line). See method 1 above.
    None
    
    ### END CODE HERE ###
    
    return cost

In [ ]:
logits = np.array([0.2,0.4,0.7,0.9])

cost = cost(logits, np.array([0,0,1,1]))
print ("cost = " + str(cost))

**모범 답안** : 

```
cost = [ 0.79813886  0.91301525  0.40318605  0.34115386]
```

### 1-4. One Hot encoding 사용하기 ###

여러분은 딥 러닝에서 여러 번 0에서 C-1까지의 숫자를 가진 y 벡터를 다루게 될 것입니다. 여기서 C는 클래스의 수입니다. 예를 들어 C가 4이면 아래 이미지와 같이 변환해야하는 다음과 같은 y 벡터가있을 수 있습니다.

<img src="arts/onehot.png" style="width:600px;height:150px;">

변환된 배열의 각 컬럼마다 정확히 1개의 값만이 "hot" 한 값(다른것들은 0인데 얘만 1)을 가지기 때문에, 이 방법을 "one hot" 인코딩이라고 부릅니다. `numpy` 라이브러리를 사용해 이와 같은 변환을 수행하려면, 몇 줄의 코드를 추가해야 했겠지만 텐서플로우에서는 한 줄의 코드면 충분합니다.
```python
tf.one_hot(labels, depth, axis)
```

**연습 문제**: 한줄의 벡터 라벨(y값) 을 받아서 전체 클래스의 수 C로 변환하는 아래 함수를 완성해보세요. `tf.one_hot()`함수를 사용하면 됩니다.

In [ ]:
# GRADED FUNCTION: one_hot_matrix

def one_hot_matrix(labels, C):
    """
    Creates a matrix where the i-th row corresponds to the ith class number and the jth column
                     corresponds to the jth training example. So if example j had a label i. Then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the labels 
    C -- number of classes, the depth of the one hot dimension
    
    Returns: 
    one_hot -- one hot matrix
    """
    
    ### START CODE HERE ###
    
    # Create a tf.constant equal to C (depth), name it 'C'. (approx. 1 line)
    C = None
    
    # Use tf.one_hot, be careful with the axis (approx. 1 line)
    one_hot_matrix = None
    
    # Create the session (approx. 1 line)
    sess = None
    
    # Run the session (approx. 1 line)
    one_hot = None
    
    # Close the session (approx. 1 line). See method 1 above.
    None
    
    ### END CODE HERE ###
    
    return one_hot

In [ ]:
labels = np.array([1,2,3,0,2,1])
one_hot = one_hot_matrix(labels, C = 4)
print ("one_hot = \n" + str(one_hot))

**모범 답안**: 

```
one_hot = 
[[ 0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.  0.]]
```

### 1-5. 0과 1로 초기화하기 ###

이제 벡터를 0과 1로 초기화하는 방법에 대해서 알아보겠습니다. 벡터를 1로 초기화하기 위해서는 텐서플로우에서 제공하는 빌트인 함수인 `tf.ones()`를 호출합니다. 0으로 초기화하는 방법은 `tf.zeros()`를 호출하면 됩니다. 이 함수들은 인자로 `shape`를 받아서 해당 shape에 맞는 형태의 배열을 주어진 값(0 혹은 1)로 초기화하여 리턴합니다.

**연습 문제**: 주어진 `shape`에 대하여 모든 값이 1로 초기화된 배열을 리턴하는 함수를 구현해보세요.
```python
tf.ones(shape)
```
함수를 이용합니다.

In [ ]:
# GRADED FUNCTION: ones

def ones(shape):
    """
    Creates an array of ones of dimension shape
    
    Arguments:
    shape -- shape of the array you want to create
        
    Returns: 
    ones -- array containing only ones
    """
    
    ### START CODE HERE ###
    
    # Create "ones" tensor using tf.ones(...). (approx. 1 line)
    ones = None
    
    # Create the session (approx. 1 line)
    sess = None
    
    # Run the session to compute 'ones' (approx. 1 line)
    ones = None
    
    # Close the session (approx. 1 line). See method 1 above.
    None
    
    ### END CODE HERE ###
    return ones

In [ ]:
print ("ones = " + str(ones([3])))

**모범 답안:**
<table> 
    <tr> 
        <td>
            <b>ones</b>
        </td>
        <td>
        [ 1.  1.  1.]
        </td>
    </tr>
</table>

## 2. 텐서플로우를 사용한 첫 번째 신경망 만들어보기 ##

이 파트에선 텐서플로우를 활용해 실제 인공 신경망을 개발합니다. 이전 파트에서 다뤘던, 텐서플로우 모델을 구현하기 위한 두 가지 단계를 기억해보세요.
- 계산 그래프(Computation graph) 만들기
- 해당 그래프를 실행하기

이제 해결해야 하는 문제를 파헤쳐봅시다.

### 2-0. 문제 정의하기 : '수화' 데이터셋 ###

어느 오후, 당신은 몇몇 친구들과 함께 컴퓨터가 당신의 수화를 해독할 수 있도록 가르쳐보기로 결정했습니다. 당신은 데이터셋을 수집하기 위해서 몇 시간 동안 하얀 벽 앞에서 사진을 찍었습니다. 이제 다음으로 해야할 일은, 말을 하지 못하는 사람과 수화를 이해하지 못하는 사람 사이의 의사소통을 도와주는 알고리즘을 구축하는 것입니다. 

- **훈련 세트**: 64x64 픽셀의 이미지 1080장입니다. 각 이미지는 0~5의 y값으로 라벨링되어있습니다. 따라서 각 y값마다 총 180개의 라벨링된 이미지 데이터가 들어있습니다.
- **테스트 세트**: 훈련 세트와 같은 규격의 이미지 120장이 들어 있습니다. 0~5의 y값으로 라벨링되어 있으며, 각 y마다 20장의 데이터가 들어 있습니다.

이 데이터는 전체 수화 언어 중 일부에 불과합니다. 실제 수화 데이터셋은 더 많은 손동작을 가지고 있을 것입니다.

아래 그림은 각 *숫자* 에 대한 예와 라벨링된 y값을 나타내는 방법에 대한 설명입니다. 이는 이미지 해상도를 64x64로 낮추기 이전의 원본 사진입니다.

<img src="images/hands.png" style="width:800px;height:350px;">

**그림 1**: 수화 데이터셋

<br>

아래 코드를 실행시켜 데이터셋을 불러오세요.

In [ ]:
# Loading the dataset
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

일부 데이터셋을 시각화해서 보고싶다면, `index` 값을 바꿔서 아래 코드 블록을 실행시켜보세요.

In [ ]:
# Example of a picture
index = 0
plt.imshow(X_train_orig[index])
print ("y = " + str(np.squeeze(Y_train_orig[:, index])))

평소와 같이 이미지 데이터셋을 펼친 다음 255로 나눠 정규화합니다. 그 후 위 그림 1번처럼 라벨링된 값을 전부 one-hot 벡터로 변환합니다. 아래 코드 블록을 실행해보세요.

In [ ]:
# Flatten the training and test images
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T
# Normalize image vectors
X_train = X_train_flatten/255.
X_test = X_test_flatten/255.
# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6)
Y_test = convert_to_one_hot(Y_test_orig, 6)

print ("number of training examples = " + str(X_train.shape[1]))
print ("number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

12288이라는 숫자는, $64 \times 64 \times 3$ 의 식으로부터 도출되었습니다. 각 이미지는 64x64 픽셀의 해상도를 가지고 있고, 나머지 3이라는 숫자는 RGB 색상을 의미합니다. 본격적인 과제를 시작하기에 앞서, 다루고자 하는 데이터셋 배열들의 형태를 충분히 숙지하세요.

**여러분의 목표**는 수화 언어를 높은 정확도로 인식할 수 있는 알고리즘을 개발하는 것입니다. 여러분은 이전에 고양이 이미지를 인식하기 위해 `numpy` 라이브러리만을 사용해서 만들었던 방식과 거의 유사한 방식으로 텐서플로우 모델을 개발할 것입니다(이번에는 `softmax` 함수로 출력값을 도출한다는 점에서 미묘한 차이점은 있을 수 있습니다). `numpy` 구현을 `tensorflow` 구현과 비교할 수있는 좋은 기회입니다. 

**모델의 전반적인 구조는** LINEAR -> RELU -> LINEAR-> RELU -> LINEAR -> SOFTMAX입니다. 최종 SIGMOID 출력층이 SOFTMAX 함수로 변환됩니다. SOFTMAX를 계산하는 층은 SIGMOID 출력값을 2개 이상의 클래스로 일반화시킵니다(분류합니다)

### 2-1. Placeholder 만들기 ###

처음으로 해야할 일은, 플레이스홀더 X와 Y를 만드는 것입니다. 이 작업은 이후 세션을 실행할 때 데이터를 넘겨주도록 합니다.

**연습 문제**: 텐서플로우의 플레이스홀더를 만드는 아래 함수를 구현하세요.

In [ ]:
# GRADED FUNCTION: create_placeholders

def create_placeholders(n_x, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, size of an image vector (num_px * num_px = 64 * 64 * 3 = 12288)
    n_y -- scalar, number of classes (from 0 to 5, so -> 6)
    
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "tf.float32"
    Y -- placeholder for the input labels, of shape [n_y, None] and dtype "tf.float32"
    
    Tips:
    - You will use None because it let's us be flexible on the number of examples you will for the placeholders.
      In fact, the number of examples during test/train is different.
    """

    ### START CODE HERE ### (approx. 2 lines)
    X = None
    Y = None
    ### END CODE HERE ###
    
    return X, Y

In [ ]:
X, Y = create_placeholders(12288, 6)
print ("X = " + str(X))
print ("Y = " + str(Y))

**Expected Output**: 

<table> 
    <tr> 
        <td>
            <b>X</b>
        </td>
        <td>
        Tensor("Placeholder_1:0", shape=(12288, ?), dtype=float32) (not necessarily Placeholder_1)
        </td>
    </tr>
    <tr> 
        <td>
            <b>Y</b>
        </td>
        <td>
        Tensor("Placeholder_2:0", shape=(6, ?), dtype=float32) (not necessarily Placeholder_2)
        </td>
    </tr>
</table>

### 2-2. 파라미터 초기화하기 ###

두 번째로 할 작업은, 텐서플로우에서 쓸 파라미터를 초기화하는 작업입니다.

**연습 문제**: 아래 텐서플로우 모델을 빌드할 때 쓸 파라미터를 초기화해보세요. 가중치를 초기화 할 때는 Xavier 초기화를, bias는 0으로 초기화합니다. 배열의 shape은 아래와 같이 하면 됩니다.예를 들어 W1 및 b1의 경우 아래 코드를 사용할 수 있습니다.
```python
W1 = tf.get_variable("W1", [25,12288], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
b1 = tf.get_variable("b1", [25,1], initializer = tf.zeros_initializer())
```
채점의 편의를 위해 `seed = 1`로 설정하세요.

In [2]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters():
    """
    Initializes parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [25, 12288]
                        b1 : [25, 1]
                        W2 : [12, 25]
                        b2 : [12, 1]
                        W3 : [6, 12]
                        b3 : [6, 1]
    
    Returns:
    parameters -- a dictionary of tensors containing W1, b1, W2, b2, W3, b3
    """
    
    tf.set_random_seed(1)                   # so that your "random" numbers match ours
        
    ### START CODE HERE ### (approx. 6 lines of code)
    W1 = None
    b1 = None
    W2 = None
    b2 = None
    W3 = None
    b3 = None
    ### END CODE HERE ###

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [ ]:
tf.reset_default_graph()
with tf.Session() as sess:
    parameters = initialize_parameters()
    print("W1 = " + str(parameters["W1"]))
    print("b1 = " + str(parameters["b1"]))
    print("W2 = " + str(parameters["W2"]))
    print("b2 = " + str(parameters["b2"]))

**모범 답안**: 
<table> 
    <tr> 
        <td>
            <b>W1</b>
        </td>
        <td>
         < tf.Variable 'W1:0' shape=(25, 12288) dtype=float32_ref >
        </td>
    </tr>
    <tr> 
        <td>
            <b>b1</b>
        </td>
        <td>
        < tf.Variable 'b1:0' shape=(25, 1) dtype=float32_ref >
        </td>
    </tr>
    <tr> 
        <td>
            <b>W2</b>
        </td>
        <td>
        < tf.Variable 'W2:0' shape=(12, 25) dtype=float32_ref >
        </td>
    </tr>
    <tr> 
        <td>
            <b>b2</b>
        </td>
        <td>
        < tf.Variable 'b2:0' shape=(12, 1) dtype=float32_ref >
        </td>
    </tr>

</table>

예상했던대로, 파라미터에 아직 값이 입력되지 않았습니다.

### 2-3. 텐서플로우를 활용한 Forward propagation ###

텐서플로우를 사용해서 정방향 연산을 구현해봅시다. 구현해야할 함수는 파라미터 딕셔너리를 인자로 받아, 정방향 계산을 수행한 결과를 리턴합니다. 텐서플로우에서 제공하는 연산용 빌트인 한수는 아래와 같습니다.

- `tf.add(...,...)` : 덧셈
- `tf.matmul(...,...)` : 행렬 곱
- `tf.nn.relu(...)` : `relu` 활성화 값 계산

**문제**: 신경망의 정방향 연산을 구현합니다. tensorflow 구현을 numpy와 비교할 수 있도록, numpy로 짜여진 코드는 어떻게 생겼는지 주석을 달았습니다. 정방향 계산은 z3에서 중지됩니다. 그 이유는 tensorflow에서 마지막 LINEAR layer 출력값이이 손실함수에 대한 입력으로 제공되기 때문입니다. 따라서 a3가 필요하지 않습니다.

In [ ]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    ### START CODE HERE ### (approx. 5 lines)              # Numpy Equivalents:
    Z1 = None                                              # Z1 = np.dot(W1, X) + b1
    A1 = None                                              # A1 = relu(Z1)
    Z2 = None                                              # Z2 = np.dot(W2, A1) + b2
    A2 = None                                              # A2 = relu(Z2)
    Z3 = None                                              # Z3 = np.dot(W3, A2) + b3
    ### END CODE HERE ###
    
    return Z3

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    X, Y = create_placeholders(12288, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    print("Z3 = " + str(Z3))

**모범 답안**: 

<table> 
    <tr> 
        <td>
            <b>Z3</b>
        </td>
        <td>
        Tensor("Add_2:0", shape=(6, ?), dtype=float32)
        </td>
    </tr>

</table>

정방향 계산 과정에서 캐시를 출력하지 않는다는 것을 알 수 있습니다. 아래에서 brackpropagation을 구현해보면 그 이유를 이해할 수 있습니다.

### 2-4. 비용 계산하기 ###

이전에도 봤듯이, 비용을 계산하는 과정은 아래 함수를 사용하면 매우 쉽게 구현할 수 있습니다.
```python
tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = ..., labels = ...))
```

**문제**: 아래의 비용 함수를 구현하세요.
- `tf.nn.softmax_cross_entropy_with_logits`의 "`logits`"및 "`labels`" 입력 파라미터는 (예제 수, num_classes)의 `shape`를 가집니다. 그러나 정방향 계산의 최종 결과값인 `A3`는 (num_classes=6, 예제 수=12288) 의 `shape`를 가지고 있습니다. 따라서 Z3과 Y를 전치한 형태로 비용 함수에 집어넣어야 합니다.
- 반면에, `tf_reduce_mean`은 모든 샘플 데이터를 더하는 함수입니다.

In [3]:
# GRADED FUNCTION: compute_cost 

def compute_cost(Z3, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    ### START CODE HERE ### (1 line of code)
    cost = None
    ### END CODE HERE ###
    
    return cost

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    X, Y = create_placeholders(12288, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
    print("cost = " + str(cost))

**모범 답안**: 

<table> 
    <tr> 
        <td>
            <b>cost</b>
        </td>
        <td>
        Tensor("Mean:0", shape=(), dtype=float32)
        </td>
    </tr>
</table>

### 2-5. Backward propagation과 파라미터 업데이트 ###

감사하게도 이 부분은 프로그래밍 프레임워크가 다 해주는 부분입니다. 각 레이어에 대한 모든 역전파 연산 및 파라미터 업데이트는 단 한줄의 코드로 구현할 수 있습니다. 이 한 줄의 코드를 모델에 추가하는 것은 매우 쉬운 일이죠.

비용함수를 계산한 이후, `optimizer` 객체를 만듭니다. `tf.session`을 실행할 때 `cost`와 함께 이 객체를 호출해야 합니다. 이 객체가 호출되면, `optimizer` 변수에 명시된 최적화 방법과 학습률로 주어진 cost에 대한 학습을 수행합니다.

예를 들어 경사 하강법의 경우 최적화 프로그램은 다음과 같습니다.
```python
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)
```

해당 `optimizer`로 최적화를 수행하라면 아래 코드를 실행하세요.
```python
_ , c = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
```

이를 통해 프레임워크는 cost -> input까지 텐서 플로우 계산 그래프를 역순으로 통과하면서 역전파를 계산합니다.

**참고**: 파이썬 프로그래밍에서, 나중에 사용할 필요가 없는 값을 저장하기 위해 종종 `_`를 "throwaway"변수로 사용합니다. 여기서 `_`는 우리가 필요로하지 않는 최적화 함수의 결과 값을 받습니다 (그리고 `c`는 비용 변수의 값을 받습니다).

### 2-6. 최종 모델 구축하기 ###

이제 앞서 구현했던 함수들을 한데 모아 모델을 만들어봅시다.

**연습 문제** : 이전에 구현했던 함수들을 호출해 모델을 구현해보세요.

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    """
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.
    
    Arguments:
    X_train -- training set, of shape (input size = 12288, number of training examples = 1080)
    Y_train -- test set, of shape (output size = 6, number of training examples = 1080)
    X_test -- training set, of shape (input size = 12288, number of training examples = 120)
    Y_test -- test set, of shape (output size = 6, number of test examples = 120)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of shape (n_x, n_y)
    ### START CODE HERE ### (1 line)
    X, Y = None
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = None
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z3 = None
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = None
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    ### START CODE HERE ### (1 line)
    optimizer = None
    ### END CODE HERE ###
    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , minibatch_cost = None
                ### END CODE HERE ###
                
                epoch_cost += minibatch_cost / minibatch_size

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per fives)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
        return parameters

아래 코드 블록을 실행시켜서 모델을 훈련시켜봅시다. 대략 5분정도의 시간이 소모됩니다. 모델이 정확히 훈련되었다면 "Cost after epoch 100"으로 표시되는 값이 1.048222가 되어야 합니다. 만약 그 값이 아니라면, 모델을 더 훈련하는 것이 의미가 없으므로 상단의 실행 정지 버튼(⬛)을 클릭하고 코드를 고쳐보세요. 만약 그 값이 정확하다면, 잠시 5분만 쉬다 오시면 됩니다.

In [ ]:
parameters = model(X_train, Y_train, X_test, Y_test)

**모범 답안**:

<table> 
    <tr> 
        <td>
            <b>Train Accuracy</b>
        </td>
        <td>
        0.999074
        </td>
    </tr>
    <tr> 
        <td>
            <b>Test Accuracy</b>
        </td>
        <td>
        0.716667
        </td>
    </tr>
</table>

놀랍게도 여러분이 구현안 알고리즘은 0~5 사이의 수화를 약 71.7%의 정확도로 구분할 수 있습니다.

**Insights**:
- 여러분의 모델은 훈련 세트에 매우 적합할 정도로 충분히 훈련되었습니다. 그러나 훈련 세트와 테스트 세트의 정확도 차이를 감안했을 때 L2 혹은 dropout 정규화 기법을 사용하여 과적합을 줄여야 할 것으로 보입니다.
- 세션을 모델 학습을 위한 코드 블록 정도으로 생각해보세요. 미니 배치에서 세션을 실행할 때마다 매개 변수가 훈련됩니다. 잘 훈련 된 매개 변수를 얻을 때까지 전체적으로 세션을 여러 번 (1500 epoch) 실행했습니다.

### 2-7. 자체 이미지를 통한 테스트 (선택 학습 / 채점 안됨) ###

과제를 모두 마치신 것을 축하드립니다! 이제 직접 여러분이 손 모양을 찍어서 모델의 output을 확인해봅시다. 아래의 방법을 따라해보세요.

1. 이 notebook 의 상단 표시줄에서 "File"을 클릭 후, "open"을 클릭하여 Cousera Hub로 이동합니다.
2. "images" 폴더에 있는 Jupyter Notebook의 디렉토리에 이미지를 추가합니다.
3. 아래 코드에서 이미지 이름을 변경합니다.
4. 코드를 실행하고, 알고리즘이 올바른 결과를 내는지 확인하세요

In [ ]:
import scipy
from PIL import Image
from scipy import ndimage

## START CODE HERE ## (PUT YOUR IMAGE NAME) 
my_image = "thumbs_up.jpg"
## END CODE HERE ##

# We preprocess your image to fit your algorithm.
fname = "images/" + my_image
image = np.array(ndimage.imread(fname, flatten=False))
image = image/255.
my_image = scipy.misc.imresize(image, size=(64,64)).reshape((1, 64*64*3)).T
my_image_prediction = predict(my_image, parameters)

plt.imshow(image)
print("Your algorithm predicts: y = " + str(np.squeeze(my_image_prediction)))

알고리즘이 잘못 분류하는 것처럼 보이지만 실제로 "좋아요"를 받을 자격이 있습니다. 그 이유는 훈련 세트에 "좋아요"가 포함되어 있지 않으므로 모델이이를 처리하는 방법을 알지 못하기 때문입니다! 이를 "불일치 데이터 분포"라고 부르며 "머신 러닝 프로젝트 구조화"에 대한 다음 과정 중 하나입니다.

**반드시 기억해야 할 것**
- 텐서플로우는 딥 러닝을 위한 프로그래밍 프레임워크입니다.
- 텐서플로우의 주된 객체 두 개는 `Tensors`와 `Operators`입니다.
- 텐서플로우를 이용해 모델을 개발할 때 아래 단계를 따릅니다.
  - Tensors(변수, Placehoder 등)와 Operations(`tf.matmul`, `tf.add` 등)를 포함하고 있는 연산 그래프를 만듭니다.
  - Session을 생성합니다.
  - Session을 초기화합니다.
  - Session을 실행시켜 연산 그래프를 동작하게 합니다.
- 위의 `model()` 함수에서도 볼 수 있듯이, 연산 그래프를 여러 회 반복해서 실행할 수 있습니다.
- 오차 역전파 및 최적화 작업은 Session이 돌아가는 도중 `optimizer` 객채에 의해 자동으로 수행됩니다.